## Train model with long sentences only

In [5]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems

### Extract long sentences from data

In [3]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

import gzip # import library for reading datasets

@registry.register_problem
class OnlineRevewProjectUSYD(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 2**15  #15 ~32768 # 13~8k

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
    # 10% evaluation data
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 9,
        }, {
            "split": problem.DatasetSplit.EVAL,
            "shards": 1,
        }]   
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        del data_dir
        del tmp_dir
        del dataset_split

        # load dataset
        dataset = "../../../data/reviews.json.gz"
        
        with gzip.open(dataset, 'rb') as f:
            
            for line in f:
                data = eval(line)               
                review = data['review']
                summary = data['summary']
                
                if summary in review:
                    continue
                    
                # only select text contains more than or equal to 384 words
                tokens = review.split()
                if len(tokens)<384: 
                    continue

                yield {
                    "inputs": review,
                    "targets": summary
                }
# import generate_problem # self-defined 'generate_problem.py' file in the same folder.

# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_v8'
TMP_DIR = './tmp' 
DATA_LOC = './data' 

# Generated training data
t2t_problem = OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR)

INFO:tensorflow:Generating vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:Processing token [historyhence] took 0.13035321235656738 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [forelockafter] took 0.26079893112182617 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [starsbinding] took 1.389662742614746 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 100619
INFO:tensorflow:Iteration 1
INFO:tensorflow:Processing token [vwbugcommercialkindaway] took 0.13007044792175293 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [normalangle] took 0.43274521827697754 seconds, consider setting Text2TextProblem.max_subtoken_length to a smal

INFO:tensorflow:vocab_size = 33039
INFO:tensorflow:Iteration 3
INFO:tensorflow:Processing token [kierkegardanother] took 0.30531835556030273 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Generating case 100000.
INFO:tensorflow:Generating case 200000.
INFO:tensorflow:Generating case 300000.
INFO:tensorflow:Generated 381865 Examples
INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:tensorflow:Data shuffled.


### Train with full length

In [6]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_long_full'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.1 # default 0.2
hparams.num_encoder_layers = 6 # default 0
hparams.num_decoder_layers = 6 # default 0
# hparams.max_input_seq_length = 512

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=20001, 
        eval_steps=100 
    )

# training
t2t_model.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_environment': 'local', '_protocol': None, '_save_checkpoints_steps': 1000, '_task_id': 0, '_model_dir': 'model_files_long_full', '_log_step_count_steps': 100, '_is_chief': True, 'use_tpu': False, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_save_summary_steps': 100, '_evaluation_master': '', '_device_fn': None, '_task_type': None, '_master': '', '_train_distribute': None, '_num_ps_replicas': 0, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f61acf13710>, '_keep_checkpoint_every_n_hours': 10000, '_num_wo

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 61006336
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_files_long_full/model.ckpt-0
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files_long_full/model.ckpt.
INFO:tensorflow:loss = 9.511127, step = 0
INFO:tensorflow:global_step/sec: 2.25914
INFO:tensorflow:loss = 8.183212, step = 100 (44.266 sec)
INFO:tensorflow:global_step/sec: 2.48103
INFO:tensorflow:loss = 7.42624, step = 200 (40.305 sec)
INFO:tensorflow:global_step/sec: 2.59666
INFO:tensorflow:loss = 7.0476007, step = 300 (38.511 sec)
INFO:tensorflow:global_step/sec: 2.5709
INFO:tensorflow:loss = 6.611059, step = 400 (38.895 sec)
INFO:tensorflow:global_step/sec: 2.52066
INFO:tens

...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T04:54:40Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_full/model.ckpt-16000
INFO:tensorflow:Running local_init_op.
INFO:tens

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T05:33:31Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_full/model.ckpt-18000
INFO:tensorflow:Running local_init_op.
INFO:tens

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T06:05:51Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_full/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tens

### Train with first 256 words

In [7]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_first256'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.1 # default 0.2
hparams.num_encoder_layers = 6 # default 0
hparams.num_decoder_layers = 6 # default 0
hparams.max_input_seq_length = 256

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model_first256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=20001, 
        eval_steps=100 
    )

# training
t2t_model_first256.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_environment': 'local', '_protocol': None, '_save_checkpoints_steps': 1000, '_task_id': 0, '_model_dir': 'model_files_first256', '_log_step_count_steps': 100, '_is_chief': True, 'use_tpu': False, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_save_summary_steps': 100, '_evaluation_master': '', '_device_fn': None, '_task_type': None, '_master': '', '_train_distribute': None, '_num_ps_replicas': 0, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f61acf137b8>, '_keep_checkpoint_every_n_hours': 10000, '_num_wor

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 61006336
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_files_first256/model.ckpt.
INFO:tensorflow:loss = 9.479578, step = 0
INFO:tensorflow:global_step/sec: 2.64494
INFO:tensorflow:loss = 8.081234, step = 100 (37.809 sec)
INFO:tensorflow:global_step/sec: 3.10808
INFO:tensorflow:loss = 7.478269, step = 200 (32.178 sec)
INFO:tensorflow:global_step/sec: 3.10522
INFO:tensorflow:loss = 6.930383, step = 300 (32.200 sec)
INFO:tensorflow:global_step/sec: 3.10747
INFO:tensorflow:loss = 6.5608726, step = 400 (32.181 sec)
INFO:tensorflow:global_step/sec: 3.10826
INFO:tensorflow:loss = 6.421112, step = 500 (32.172 sec)
INFO:tensorflow:global_step/sec: 3.1044
INFO:tensorflow:loss = 6.5723214, step = 600 (32.214 sec)
INFO:tensorflow:global_step/sec: 3.10482
INFO:tensorflow:loss = 6.390116, step = 700 (3

...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T08:00:28Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_first256/model.ckpt-16000
INFO:tensorflow:Running local_init_op.
INFO:tenso

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T08:12:11Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_first256/model.ckpt-18000
INFO:tensorflow:Running local_init_op.
INFO:tenso

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T08:23:56Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_first256/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tenso

### Train with last 256 words

In [8]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

import gzip # import library for reading datasets

@registry.register_problem
class OnlineRevewProjectUSYD_last256(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 2**15  #15 ~32768 # 13~8k

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
    # 10% evaluation data
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 9,
        }, {
            "split": problem.DatasetSplit.EVAL,
            "shards": 1,
        }]   
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        del data_dir
        del tmp_dir
        del dataset_split

        # load dataset
        dataset = "../../../data/reviews.json.gz"
        
        with gzip.open(dataset, 'rb') as f:
            
            for line in f:
                data = eval(line)
                # only select text contains more than or equal to 384 words
                review = data['review']
                summary = data['summary']
                
                if summary in review:
                    continue

                tokens = review.split()
                if len(tokens)<384: 
                    continue
                else: 
                    selected = tokens[-256:]
                    review = ' '.join(selected)

                yield {
                    "inputs": review,
                    "targets": data['summary']
                }
# import generate_problem # self-defined 'generate_problem.py' file in the same folder.

# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_last256'
TMP_DIR = './tmp_last256' 
DATA_LOC = './data_last256' 

# Generated training data
t2t_problem = OnlineRevewProjectUSYD_last256(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR)

INFO:tensorflow:Generating vocab file: ./data_last256/vocab.online_revew_project_usyd_last256.32768.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:Processing token [lightsyou] took 1.7802941799163818 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [partners8217] took 3.947747230529785 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [paneldashboard] took 6.3305511474609375 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [pp102w24] took 14.582066297531128 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:Processing token [distortionabout] took 33.500617027282715 seconds, consider setting Text2TextProblem.max_subtoken_length to a smaller value.
INFO:tensorflow:vocab_size = 58015
INFO:tensorflow:Ite

In [10]:
# select the transformer model
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_last256'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.1 # default 0.2
hparams.num_encoder_layers = 6 # default 0
hparams.num_decoder_layers = 6 # default 0
hparams.max_input_seq_length = 256

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model_last256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data_last256', 
        train_steps=20001, 
        eval_steps=100 
    )

# training
t2t_model_last256.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_environment': 'local', '_protocol': None, '_save_checkpoints_steps': 1000, '_task_id': 0, '_model_dir': 'model_files_last256', '_log_step_count_steps': 100, '_is_chief': True, 'use_tpu': False, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_save_summary_steps': 100, '_evaluation_master': '', '_device_fn': None, '_task_type': None, '_master': '', '_train_distribute': None, '_num_ps_replicas': 0, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f616e230518>, '_keep_checkpoint_every_n_hours': 10000, '_num_work

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32926_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32926_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32926_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 60978176
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32926_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32926_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32926_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T10:42:50Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_last256/model.ckpt-16000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32926_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32926_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32926_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T10:54:33Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_last256/model.ckpt-18000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32926_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32926_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32926_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T11:06:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_last256/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensor

# Hyparams - num_heads 

In [13]:
# define the directory for generated data
USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_v8'
TMP_DIR = './tmp' 
DATA_LOC = './data' 

# Generated training data
t2t_problem = OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR)
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_long_16head'

hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.1 
hparams.num_encoder_layers = 6 
hparams.num_decoder_layers = 6 
# hparams.num_hidden_layers =12 # default 6
hparams.num_heads = 16 # default 8
print(hparams)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

t2t_model_head = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=t2t_problem, #PROBLEM,
        data_dir='./data', 
        train_steps=20001, 
        eval_steps=100 
    )

# training
t2t_model_head.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords
INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd-unshuffled-train-00000-of-00009', './data/online_revew_project_usyd-unshuffled-train-00001-of-00009', './data/online_revew_project_usyd-unshuffled-train-00002-of-00009', './data/online_revew_project_usyd-unshuffled-train-00003-of-00009', './data/online_revew_project_usyd-unshuffled-train-00004-of-00009', './data/online_revew_project_usyd-unshuffled-train-00005-of-00009', './data/online_revew_project_usyd-unshuffled-train-00006-of-00009', './data/online_revew_project_usyd-unshuffled-train-00007-of-00009', './data/online_revew_project_usyd-unshuffled-train-00008-of-00009', './data/online_revew_project_usyd-unshuffled-dev-00000-of-00001']
INFO:tensorflow:Skipping shuffle because output files exist
[('activation_dtype', 'float32'), ('add_relative_to_values', False), ('attention_dropout', 0.1), ('attenti

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 61006336
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T15:41:28Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_16head/model.ckpt-16000
INFO:tensorflow:Running local_init_op.
INFO:te

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T16:30:38Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_16head/model.ckpt-18000
INFO:tensorflow:Running local_init_op.
INFO:te

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_32981_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32981_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_32981_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-27T17:17:44Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_long_16head/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:te